In [13]:
import os, torch,json, random, numpy as np
import pandas as pd
import shutil # shutil 모듈을 사용하여 파일 및 디렉토리 복사, 이동, 삭제 등을 수행할 수 있습니다.
DIR1 = './results/temp_1'
DIR2 = './results/temp_2'
DIR = '_'.join(DIR1.split('_')[:-1])
DIR = DIR.replace('temp', 'tratras')
random.seed(2025)
np.random.seed(2025)
torch.manual_seed(2025)

In [ ]:
### 1. 새로운 디렉토리 
if not os.path.exists(DIR):
    os.makedirs(DIR)
else:
    # 디렉토리가 이미 존재하는 경우 예외 처리
    shutil.rmtree(DIR)
    print(f"Directory {DIR} already exists. Removing it and creating a new one.")
    os.makedirs(DIR)

### 2. checkpoint 및 log, eval_results 디렉토리 복사
os.makedirs(os.path.join(DIR, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(DIR, 'logs'), exist_ok=True)
os.makedirs(os.path.join(DIR, 'eval_results'), exist_ok=True)
os.makedirs(os.path.join(DIR, 'plots'), exist_ok=True)

In [3]:
### dir 보기 
"""
---DIR1
    ---checkpoints
    ---eval_results
    ---logs (folder+file)
"""
### 3. checkpoint 파일 이동
def copy_files(init_dir, to_dir):
    # checkpoints
    init_checkpoint_dir = os.path.join(init_dir, 'checkpoints')
    to_checkpoint_dir = os.path.join(to_dir, 'checkpoints')
    checkpoints = os.listdir(init_checkpoint_dir) # checkpoint dir의 파일 리스트
    for checkpoint in checkpoints:
        init_checkpoint_path = os.path.join(init_checkpoint_dir, checkpoint)
        to_checkpoint_path = os.path.join(to_checkpoint_dir, checkpoint)
        if os.path.isfile(init_checkpoint_path):
            shutil.copy2(init_checkpoint_path, to_checkpoint_path) # copy2는 파일의 메타데이터(예: 수정 시간)를 유지하면서 복사합니다.
        else:
            raise FileNotFoundError(f"{init_checkpoint_path} is not a file.")
    
    # logs
    init_logs_dir = os.path.join(init_dir, 'logs')
    to_logs_dir = os.path.join(to_dir, 'logs')
    logs = os.listdir(init_logs_dir) # logs dir의 파일 리스트 --> 폴더이다. 
    for log in logs:
        init_log_path = os.path.join(init_logs_dir, log)
        to_log_path = os.path.join(to_logs_dir, log)
        if os.path.isdir(init_log_path):
            shutil.copytree(init_log_path, to_log_path) # copytree는 디렉토리와 그 안의 모든 파일을 복사합니다.
        else:
            raise FileNotFoundError(f"{init_log_path} is not a directory.")
    return None
def concat_config(init_dir1, init_dir2,output_dir):
    # Find the last two JSON files in the directory
    config_path = os.path.join(init_dir1, 'model_config.json')
    config_path2 = os.path.join(init_dir2, 'model_config.json')
    # 두 개의 config 파일을 병합하는 함수
    with open(config_path, 'r') as f1, open(config_path2, 'r') as f2:
        data1 = f1.read()
        data2 = f2.read()
    dict1 = json.loads(data1)
    dict2 = json.loads(data2)
    merged_dict = {**dict1, **dict2}
    data1 = json.dumps(merged_dict, indent=4)
    # 새로운 config 파일 경로
    to_config_path = os.path.join(output_dir, f"model_config.json")
    with open(to_config_path, 'w') as f:
        f.write(data1)
    return None
copy_files(DIR1, DIR)
copy_files(DIR2, DIR)
concat_config(DIR1, DIR2, DIR)
print("All files copied successfully.")


All files copied successfully.


### 모델 불러오기 

In [1]:
import torch.nn as nn
from model import model_classes
DIR = './results/pos2'
model_classes = model_classes
print("Model classes loaded successfully for len(model_classes):", len(model_classes))
print("Model classes:", model_classes.keys())
    

Model classes loaded successfully for len(model_classes): 12
Model classes: dict_keys(['DropCaeDenoisingElu', 'DropCaeElu', 'MultiDecoderFusionAE', 'PatchEmbedding', 'ResidualGate', 'SimpleSwinBlock', 'SwinGateAEC1ReLU', 'SwinGateAEC2Leaky', 'SwinGateAEC3ELU', 'SwinGateAEC4GELULN', 'SwinLiteElu', 'SwinLiteGatedLeaky'])


### DATA LOAD

In [2]:
# Fashion MNIST dataset
import torchvision.transforms as transforms
import torch 
from torchvision import datasets
# Fashion MNIST dataset
trainset = datasets.FashionMNIST(
    root      = './.data/', train = True,
    download  = True,
    transform = transforms.Compose([
    transforms.ToTensor(),
    ]))

testset = datasets.FashionMNIST(
    root      = './.data/', train     = False,
    download  = True,
    transform = transforms.Compose([
    transforms.ToTensor(),
    ]))


In [3]:
SELECT_NORMAL = 2 # Set 2 class as train dataset.
trainset.data = trainset.data[trainset.targets == SELECT_NORMAL]
trainset.targets = trainset.targets[trainset.targets == SELECT_NORMAL] # Set 2 class as train dataset.

test_label = [2,4,6] # Define actual test class that we use
actual_testdata = torch.isin(testset.targets, torch.tensor(test_label))
testset.data = testset.data[actual_testdata]
testset.targets = testset.targets[actual_testdata]

test_loader = torch.utils.data.DataLoader(
    dataset     = testset, batch_size  = 1,
    shuffle     = False,num_workers = 2)

train_data_size = len(trainset)
test_data_size = len(testset)

print("Train data size:", train_data_size, "Test data size:", test_data_size)

Train data size: 6000 Test data size: 3000


In [4]:
# 데이터셋을 먼저 train과 val로 나누고, train에 대해서만 증강을 적용
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
# valset은 증강을 적용하지 않음
# 데이터셋을 먼저 train과 val로 나누기
n_val = int(len(trainset) * 0.2)
n_train = len(trainset) - n_val

augset, valset = torch.utils.data.random_split(trainset, [n_train, n_val], generator=torch.Generator().manual_seed(2025))

# data size check
print("Train data size:", len(augset),"Val data size:", len(valset),"Test data size:", len(testset))

Train data size: 4800 Val data size: 1200 Test data size: 3000


In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset     = augset, batch_size  = 1,
    shuffle     = False,num_workers = 0)
val_loader = torch.utils.data.DataLoader(
    dataset     = valset, batch_size  = 1,
    shuffle     = False,num_workers = 0)
print("Train data size:", len(train_loader), "Val data size:", len(val_loader), "Test data size:", len(test_loader))


Train data size: 4800 Val data size: 1200 Test data size: 3000


### EVAL ALL

In [6]:
from loss.losses import FlexibleLoss
inference_loss_configs = {
    # ✅ 기본형: MSE
    "MSE": FlexibleLoss(
        mode="mse",
        loss_weights={"mse": 1.0},
        reduction="none"
    ),

    # ✅ 중심 영역 복원 실패 감지
    "fft" : FlexibleLoss(
        mode="fft",
        loss_weights={"fft": 1.0},
        reduction="none"
    ),
    # ✅ 엣지 감지 (고주파 정보 손실에 민감)
    "MSE+Edge": FlexibleLoss(
        mode="mse+edge",
        loss_weights={
            "mse": 0.7,
            "edge": 0.3
        },
        reduction="none"
    ),

    # ✅ 구조 + 텍스처 복원 실패 반영
    "MS-ssim": FlexibleLoss(
        mode="ms-ssim",
        loss_weights={"ms-ssim": 1.0},
        reduction="none"
    ),
}


In [9]:
from eval_module.module import GridEvaluator
import random
import numpy as np, os
# Set random seed for reproducibility
random.seed(2025)
np.random.seed(2025)
torch.manual_seed(2025)
grid = GridEvaluator(
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    checkpoint_dir=os.path.join(DIR, 'checkpoints'),
    plot_dir=os.path.join(DIR, 'plots'),
    inference_dir=os.path.join(DIR, 'eval_results'),
    model_map=model_classes,
    loss_fns=inference_loss_configs,
    device=DEVICE,
)
grid.run(percentile=0.6)

Evaluating model: SwinGateAEC1ReLU_C1 with loss: mse
[정상 개수, 예측 정상개수] {0: 1000, 1: 2000}, [비정상 개수, 예측 비정상개수] {0: 1381, 1: 1619}
 ROC-AUC: 0.6373 | PR-AUC: 0.7375 | F1: 0.6704 | ACC: 0.6023 | Threshold: 0.0005
--------------------------------------------------
[정상 개수, 예측 정상개수] {0: 1000, 1: 2000}, [비정상 개수, 예측 비정상개수] {0: 1356, 1: 1644}
 ROC-AUC: 0.6434 | PR-AUC: 0.7414 | F1: 0.6756 | ACC: 0.6060 | Threshold: 0.5495
--------------------------------------------------
[정상 개수, 예측 정상개수] {0: 1000, 1: 2000}, [비정상 개수, 예측 비정상개수] {0: 1396, 1: 1604}
 ROC-AUC: 0.6122 | PR-AUC: 0.7297 | F1: 0.6576 | ACC: 0.5887 | Threshold: 0.0172
--------------------------------------------------
[정상 개수, 예측 정상개수] {0: 1000, 1: 2000}, [비정상 개수, 예측 비정상개수] {0: 1013, 1: 1987}
 ROC-AUC: 0.7261 | PR-AUC: 0.8020 | F1: 0.7775 | ACC: 0.7043 | Threshold: 0.0090
--------------------------------------------------
Evaluation complete for model: SwinGateAEC1ReLU_C1 with loss: mse
Evaluating model: SwinGateAEC1ReLU_C2 with loss: msea

In [11]:
grid.save_results(output_dir=os.path.join(DIR, 'eval_results'))

In [10]:
import pandas as pd
import os
df = pd.read_csv(os.path.join(DIR, 'eval_results', 'evaluation_results.csv'))
df.sort_values(by='acc', ascending=False, inplace=True)
df[['model_name', 'train_loss_name','loss_type', 'acc','path']]

FileNotFoundError: [Errno 2] No such file or directory: './results/pos2\\eval_results\\evaluation_results.csv'